In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Customers").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/08 12:50:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression

24/07/08 12:50:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
df = spark.read.csv("Ecommerce_Customers.csv", inferSchema = True, header = True)

In [18]:
print(df.show())

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [20]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [21]:
featureassembler = VectorAssembler(inputCols = ["Avg Session Length","Time on App","Time on Website","Length of Membership"], outputCol = "independant col")

In [23]:
outdf = featureassembler.transform(df)

In [25]:
finaldf = outdf.select(["independant col","Yearly Amount Spent"])

In [32]:
traindf,testdf = finaldf.randomSplit([0.8,0.2])

In [30]:
regressor = LinearRegression(featuresCol = 'independant col' , labelCol = 'Yearly Amount Spent')
regressor = regressor.fit(traindf)

24/07/08 13:38:33 WARN Instrumentation: [4ae2e140] regParam is zero, which might cause numerical instability and overfitting.
24/07/08 13:38:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/07/08 13:38:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/07/08 13:38:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [33]:
pred_res = regressor.evaluate(testdf)

In [36]:
pred_res.predictions.show()

+--------------------+-------------------+------------------+
|     independant col|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.83643267,13.1...|        467.5019004| 470.2218652511524|
|[31.06132516,12.3...|        487.5554581| 492.3227432792139|
|[31.12397435,12.3...|        486.9470538|507.03170852385665|
|[31.12809005,13.2...|        557.2526867| 563.2765533721415|
|[31.3123496,11.68...|         463.591418|443.88018269145164|
|[31.36621217,11.1...|        430.5888826|425.87968617352453|
|[31.44744649,10.1...|        418.6027421|  425.531074569971|
|[31.57413802,12.9...|        544.4092722| 557.5458343565317|
|[31.66104982,11.3...|        416.3583536| 416.4693957491363|
|[31.81248256,10.8...|         392.810345| 395.5081433770415|
|[31.91207593,11.7...|        387.5347163| 388.7374902970796|
|[32.00475302,11.3...|        463.7459811| 462.9420198075891|
|[32.00850452,12.0...|         443.197221|451.19194391885685|
|[32.030